<a href="https://colab.research.google.com/github/rocioromera911/Portafolio_RR/blob/main/Apparel_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apparel Classifier - EfficientNetB0 - Colab Notebook
Entrena un modelo de clasificación de ropa con máxima precisión usando Transfer Learning.

In [ ]:
# ✅ Paso 1: Instalar librerías necesarias
!pip install -q tensorflow pandas matplotlib scikit-learn opencv-python

In [ ]:
# ✅ Paso 2: Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# ✅ Paso 3: Montar Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ✅ Paso 4: Configurar paths
base_path = '/content/drive/MyDrive/apparel_dataset'

In [ ]:
# ✅ Paso 5: Preparar datos
df = pd.read_csv(os.path.join(base_path, 'train.csv'))
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

In [ ]:
# ✅ Generadores
img_size = 224
batch_size = 32
epochs = 30
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=25, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
train_gen = train_datagen.flow_from_dataframe(train_df, directory=os.path.join(base_path, 'train'), x_col='image_id', y_col='label', target_size=(img_size, img_size), class_mode='categorical', batch_size=batch_size)
val_gen = val_datagen.flow_from_dataframe(val_df, directory=os.path.join(base_path, 'train'), x_col='image_id', y_col='label', target_size=(img_size, img_size), class_mode='categorical', batch_size=batch_size)

In [ ]:
# ✅ Modelo
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# ✅ Callbacks y entrenamiento
callbacks = [EarlyStopping(patience=5, restore_best_weights=True), ReduceLROnPlateau(patience=3, factor=0.5)]
history = model.fit(train_gen, validation_data=val_gen, epochs=epochs, callbacks=callbacks)

In [ ]:
# ✅ Gráfico
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.legend()
plt.title('Accuracy')
plt.show()

In [ ]:
# ✅ Predicciones
test_files = os.listdir(os.path.join(base_path, 'test'))
test_df = pd.DataFrame({'image_id': test_files})
test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_dataframe(test_df, directory=os.path.join(base_path, 'test'), x_col='image_id', y_col=None, target_size=(img_size, img_size), class_mode=None, batch_size=batch_size, shuffle=False)
preds = model.predict(test_gen)
labels = np.argmax(preds, axis=1)
submission = pd.DataFrame({'Image_ID': test_df['image_id'], 'Label': labels})
submission.to_csv('submission.csv', index=False)